In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import random
import numpy as np
import glob
import os
import itertools

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [ ]:
class CycleGANDataset(Dataset):
    def __init__(self, files_A, files_B, transform=None):
        self.files_A = glob.glob(files_A)
        self.files_B = glob.glob(files_B)
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
    def __getitem__(self, ix):
        try:
            fileA = Image.open(self.files_A[ix % len(self.files_A)]).convert('RGB')
            fileB = Image.open(self.files_B[ix % len(self.files_B)]).convert('RGB')
        except Exception as e:
            print(f"Error loading image: {e}")
            return None, None

        if self.transform:
            fileA = self.transform(fileA)
            fileB = self.transform(fileB)
        
        return fileA, fileB
    
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))
    
    def collate_fn(self, batch):
        filesA, filesB = zip(*batch)
        filesA = torch.stack([img for img in filesA if img is not None]).to(device).float()
        filesB = torch.stack([img for img in filesB if img is not None]).to(device).float()
        return filesA, filesB

In [ ]:
train_dataset = CycleGANDataset('/kaggle/input/apple2orange-dataset/trainA/*', '/kaggle/input/apple2orange-dataset/trainB/*')
test_dataset = CycleGANDataset('/kaggle/input/apple2orange-dataset/testA/*', '/kaggle/input/apple2orange-dataset/testB/*')

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=train_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=test_dataset.collate_fn)

# Network Initialization

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# Generator

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, num_blocks=9):
        super().__init__()
        
        self.pad1 = nn.ReflectionPad2d(3)
        self.conv1 = nn.Conv2d(3, 64, 7)
        self.norm1 = nn.InstanceNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(64, 128, 3, stride=2, padding=1)
        self.norm2 = nn.InstanceNorm2d(128)
        self.relu2 = nn.ReLU(inplace=True)
        
        self.conv3 = nn.Conv2d(128, 256, 3, stride=2, padding=1)
        self.norm3 = nn.InstanceNorm2d(256)
        self.relu3 = nn.ReLU(inplace=True)
        
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(num_blocks)]
        )
        
        self.upsample1 = nn.Upsample(scale_factor=2)
        self.conv4 = nn.Conv2d(256, 128, 3, stride=1, padding=1)
        self.norm4 = nn.InstanceNorm2d(128)
        self.relu4 = nn.ReLU(inplace=True)
        
        self.upsample2 = nn.Upsample(scale_factor=2)
        self.conv5 = nn.Conv2d(128, 64, 3, stride=1, padding=1)
        self.norm5 = nn.InstanceNorm2d(64)
        self.relu5 = nn.ReLU(inplace=True)
        
        self.pad2 = nn.ReflectionPad2d(3)
        self.conv6 = nn.Conv2d(64, 3, 7)
        self.tanh = nn.Tanh()

        self.apply(weights_init_normal)
    
    def forward(self, x):
        x = self.pad1(x)
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        
        x = self.residual_blocks(x)
        
        x = self.upsample1(x)
        x = self.conv4(x)
        x = self.norm4(x)
        x = self.relu4(x)
        
        x = self.upsample2(x)
        x = self.conv5(x)
        x = self.norm5(x)
        x = self.relu5(x)
        
        x = self.pad2(x)
        x = self.conv6(x)
        x = self.tanh(x)
        
        return x

In [ ]:
G_apple = Generator().to(device)
G_orange = Generator().to(device)

# Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        channels, height, width = 3, 256, 256

        self.conv1 = nn.Conv2d(channels, 64, 4, stride=2, padding=1)
        self.leaky_relu1 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv2 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.norm2 = nn.InstanceNorm2d(128)
        self.leaky_relu2 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv3 = nn.Conv2d(128, 256, 4, stride=2, padding=1)
        self.norm3 = nn.InstanceNorm2d(256)
        self.leaky_relu3 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv4 = nn.Conv2d(256, 512, 4, stride=2, padding=1)
        self.norm4 = nn.InstanceNorm2d(512)
        self.leaky_relu4 = nn.LeakyReLU(0.2, inplace=True)
        
        self.zero_pad = nn.ZeroPad2d((1, 0, 1, 0))
        self.conv5 = nn.Conv2d(512, 1, 4, padding=1)
        
        self.apply(weights_init_normal)

    def forward(self, img):
        x = self.conv1(img)
        x = self.leaky_relu1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.leaky_relu2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.leaky_relu3(x)
        
        x = self.conv4(x)
        x = self.norm4(x)
        x = self.leaky_relu4(x)
        
        x = self.zero_pad(x)
        x = self.conv5(x)
        
        return x

In [ ]:
D_apple = Discriminator().to(device)
D_orange = Discriminator().to(device)

# Train

In [ ]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

optimizer_G = torch.optim.Adam(
    itertools.chain(G_apple.parameters(), G_orange.parameters()), lr=0.0002, betas=(0.5, 0.999)
)
optimizer_D_apple = torch.optim.Adam(D_apple.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_orange = torch.optim.Adam(D_orange.parameters(), lr=0.0002, betas=(0.5, 0.999))

lambda_cyc, lambda_id = 10.0, 5.0

In [ ]:
g_loss_s = []
d_loss_s = []

In [ ]:
os.makedirs('./cyclegan')

In [ ]:
for epoch in range(3):
    for i, batch in enumerate(train_dataloader):
        real_apple, real_orange = batch
        
        G_apple.eval()
        G_orange.eval()
        optimizer_G.zero_grad()
        
        loss_id_apple = criterion_identity(G_orange(real_apple), real_apple)
        loss_id_orange = criterion_identity(G_apple(real_orange), real_orange)
        loss_identity = (loss_id_apple + loss_id_orange) / 2
        
        fake_orange = G_apple(real_apple)
        loss_g_apple = criterion_GAN(D_orange(fake_orange), torch.Tensor(np.ones((len(real_apple), 1, 16, 16))).to(device))
        fake_apple = G_orange(real_orange)
        loss_g_orange = criterion_GAN(D_apple(fake_apple), torch.Tensor(np.ones((len(real_apple), 1, 16, 16))).to(device))
        loss_g = (loss_g_apple + loss_g_orange) / 2
        
        recov_apple = G_orange(fake_orange)
        loss_cycle_apple = criterion_cycle(recov_apple, real_apple)
        recov_orange = G_apple(fake_apple)
        loss_cycle_orange = criterion_cycle(recov_orange, real_orange)
        loss_cycle = (loss_cycle_apple + loss_cycle_orange) / 2
        
        loss_generator = loss_g + lambda_cyc * loss_cycle + lambda_id * loss_identity
        g_loss_s.append(loss_generator)
        loss_generator.backward()
        optimizer_G.step()
        
        optimizer_D_apple.zero_grad()
        loss_real_apple = criterion_GAN(D_apple(real_apple), torch.Tensor(np.ones((len(real_apple), 1, 16, 16))).to(device))
        loss_fake_apple = criterion_GAN(D_apple(fake_apple.detach()), torch.Tensor(np.zeros((len(real_apple), 1, 16, 16))).to(device))
        loss_D_apple = (loss_real_apple + loss_fake_apple) / 2
        loss_D_apple.backward()
        optimizer_D_apple.step()
        
        optimizer_D_orange.zero_grad()
        loss_real_orange = criterion_GAN(D_orange(real_orange), torch.Tensor(np.ones((len(real_orange), 1, 16, 16))).to(device))
        loss_fake_orange = criterion_GAN(D_orange(fake_orange.detach()), torch.Tensor(np.zeros((len(real_orange), 1, 16, 16))).to(device))
        loss_D_orange = (loss_real_orange + loss_fake_orange) / 2
        loss_D_orange.backward()
        optimizer_D_orange.step()
        
        loss_discriminator = (loss_D_apple + loss_D_orange) / 2
        d_loss_s.append(loss_discriminator)
        
        if i % 100 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 10, i, len(train_dataloader), loss_discriminator, loss_generator))

            G_apple.eval()
            G_orange.eval()
            
            data_apple = next(iter(train_dataloader))
            data_orange = next(iter(test_dataloader))
            
            real_apple, _ = data_apple
            real_orange, _ = data_orange
            
            fake_orange = G_apple(real_apple)
            fake_apple = G_orange(real_orange)
            
            real_apple = make_grid(real_apple, nrow=1, normalize=True)
            real_orange = make_grid(real_orange, nrow=1, normalize=True)
            fake_apple = make_grid(fake_apple, nrow=1, normalize=True)
            fake_orange = make_grid(fake_orange, nrow=1, normalize=True)
            
            image_grid = torch.cat((real_apple, fake_orange, real_orange, fake_apple), 2)
            plt.figure()
            plt.imshow(image_grid.detach().cpu().permute(1, 2, 0).numpy())
            plt.show()
            save_image(image_grid, f'cyclegan/epoch_{epoch:03d}.png', normalize=True)

# Results

In [ ]:
g_loss_s = [g_loss_.cpu().detach().item() for g_loss_ in g_loss_s]
d_loss_s = [d_loss_.cpu().detach().item() for d_loss_ in d_loss_s]

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(g_loss_s)
plt.plot(d_loss_s)
plt.title("Loss Curve")
plt.legend(["Generator Loss", "Discriminator Loss"])
plt.show()

In [ ]:
from IPython import display
display.Image('cyclegan/epoch_002.png')